In [11]:
import pickle
import numpy

from sklearn import cross_validation
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectPercentile, f_classif
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [2]:
def preprocess(words_file = "data/word_data.pkl", authors_file="data/email_authors.pkl"):
    """ 
        this function takes a pre-made list of email texts (by default word_data.pkl)
        and the corresponding authors (by default email_authors.pkl) and performs
        a number of preprocessing steps:
            -- splits into training/testing sets (10% testing)
            -- vectorizes into tfidf matrix
            -- selects/keeps most helpful features

        after this, the feaures and labels are put into numpy arrays, which play nice with sklearn functions

        4 objects are returned:
            -- training/testing features
            -- training/testing labels

    """

    ### the words (features) and authors (labels), already largely preprocessed
    ### this preprocessing will be repeated in the text learning mini-project
    authors_file_handler = open(authors_file, "rb")
    authors = pickle.load(authors_file_handler)
    authors_file_handler.close()

    words_file_handler = open(words_file, "rb")
    word_data = pickle.load(words_file_handler)
    words_file_handler.close()

    ### test_size is the percentage of events assigned to the test set (remainder go into training)
    features_train, features_test, labels_train, labels_test = cross_validation.train_test_split(word_data, authors, test_size=0.1, random_state=42)



    ### text vectorization--go from strings to lists of numbers
    vectorizer = TfidfVectorizer(sublinear_tf=True, max_df=0.5,
                                 stop_words='english')
    features_train_transformed = vectorizer.fit_transform(features_train)
    features_test_transformed  = vectorizer.transform(features_test)



    ### feature selection, because text is super high dimensional and 
    ### can be really computationally chewy as a result
    selector = SelectPercentile(f_classif, percentile=10)
    selector.fit(features_train_transformed, labels_train)
    features_train_transformed = selector.transform(features_train_transformed).toarray()
    features_test_transformed  = selector.transform(features_test_transformed).toarray()

    ### info on the data
    print("no. of Chris training emails:", sum(labels_train))
    print("no. of Sara training emails:", len(labels_train)-sum(labels_train))
    
    return features_train_transformed, features_test_transformed, labels_train, labels_test

## Split data into training and testing

In [18]:
%time features_train, features_test, labels_train, labels_test = preprocess()

no. of Chris training emails: 7936
no. of Sara training emails: 7884
CPU times: user 2.91 s, sys: 1.03 s, total: 3.94 s
Wall time: 3.93 s


In [19]:
len(features_train)

15820

In [20]:
len(features_test)

1758

## instructions
Import, create, train and make predictions with the sklearn SVC classifier. When creating the classifier, use a linear kernel (if you forget this step, you will be unpleasantly surprised by how long the classifier takes to train). What is the accuracy of the classifier?

In [21]:
features_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [17]:
%%time
clf = SVC(kernel="linear")
clf.fit(features_train,labels_train)
pred = clf.predict(features_test)
acc = accuracy_score(pred, labels_test)
print(i)

1


## with 1% of data

In [22]:
features_train = features_train[:len(features_train)//100] 
labels_train = labels_train[:len(labels_train)//100] 

In [23]:
len(features_train)

158

In [24]:
len(labels_train)

158

In [26]:
%%time
clf = SVC(kernel="linear")
clf.fit(features_train,labels_train)
pred = clf.predict(features_test)
acc = accuracy_score(pred, labels_test)
print(acc)

0.8845278725824801
CPU times: user 1.17 s, sys: 15.6 ms, total: 1.19 s
Wall time: 1.16 s


## Change kernel

In [27]:
%%time
clf = SVC(kernel="rbf",C=1)
clf.fit(features_train,labels_train)
pred = clf.predict(features_test)
acc = accuracy_score(pred, labels_test)
print(acc)

0.6160409556313993
CPU times: user 1.31 s, sys: 0 ns, total: 1.31 s
Wall time: 1.32 s


## change C value and compare

### `C=10`

In [28]:
%%time
clf = SVC(kernel="rbf",C=10)
clf.fit(features_train,labels_train)
pred = clf.predict(features_test)
acc = accuracy_score(pred, labels_test)
print(acc)

0.6160409556313993
CPU times: user 1.31 s, sys: 15.6 ms, total: 1.33 s
Wall time: 1.32 s


### `C=100`

In [29]:
%%time
clf = SVC(kernel="rbf",C=100)
clf.fit(features_train,labels_train)
pred = clf.predict(features_test)
acc = accuracy_score(pred, labels_test)
print(acc)

0.6160409556313993
CPU times: user 1.31 s, sys: 0 ns, total: 1.31 s
Wall time: 1.32 s


### `C=1000`

In [30]:
%%time
clf = SVC(kernel="rbf",C=1000)
clf.fit(features_train,labels_train)
pred = clf.predict(features_test)
acc = accuracy_score(pred, labels_test)
print(acc)

0.8213879408418657
CPU times: user 1.27 s, sys: 15.6 ms, total: 1.28 s
Wall time: 1.26 s


## full data again
- kernel rbf
- C = 1000

In [31]:
%time features_train, features_test, labels_train, labels_test = preprocess()

no. of Chris training emails: 7936
no. of Sara training emails: 7884
CPU times: user 2.75 s, sys: 953 ms, total: 3.7 s
Wall time: 3.7 s


In [32]:
%%time
clf = SVC(kernel="rbf",C=1000)
clf.fit(features_train,labels_train)
pred = clf.predict(features_test)
acc = accuracy_score(pred, labels_test)
print(acc)

0.9829351535836177
CPU times: user 4min 39s, sys: 750 ms, total: 4min 40s
Wall time: 4min 40s


In [33]:
%%time
clf = SVC(kernel="rbf",C=100)
clf.fit(features_train,labels_train)
pred = clf.predict(features_test)
acc = accuracy_score(pred, labels_test)
print(acc)

0.9550625711035268
CPU times: user 11min 29s, sys: 438 ms, total: 11min 29s
Wall time: 11min 30s
